# Point- 1. Creating a new Notebook for this assignment.

In [3]:
!pip install wikipedia
!pip install lxml

     |████████████████████████████████| 122kB 6.4MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/87/2a/18/4e471fd96d12114d16fe4a446d00c3b38fb9efcb744bd31f4a
Successfully built wikipedia
     |████████████████████████████████| 5.5MB 9.0MB/s eta 0:00:01


In [4]:
import pandas as pd
import wikipedia as wp
import lxml

### Point- 2. Import Wikipedia table and create a List of Dictionary
#### Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

In [5]:
import requests
from bs4 import BeautifulSoup

URL = ("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
res = requests.get(URL).text
soup = BeautifulSoup(res,'lxml')
datalist = []
for items in soup.find('table', class_='wikitable').find_all('tr')[1::1]:
    data = items.find_all(['th','td'])
    try:
        PostalCode =  str.rstrip(data[0].text)
        Borough = str.rstrip(data[1].text)
        Neighborhood = str.rstrip(data[2].text)
    except IndexError:pass
    datalist.append({'PostalCode':PostalCode,'Borough':Borough,'Neighborhood':Neighborhood})

### Point- 3. To create the above dataframe:
#### The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [6]:
df = pd.DataFrame(datalist)
df

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


#### Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [7]:
df_assigned = df[df['Borough'] != "Not assigned"]
df_assigned

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


#### More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [8]:
joined_df = df_assigned.pivot_table(index=['PostalCode','Borough'],aggfunc=','.join).reset_index()
joined_df

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."


#### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [9]:
df_notassigned = df[df['Neighborhood'] == "Not assigned"]
df_notassigned['Neighborhood'] = df_notassigned['Borough']
df_notassigned

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
7,M8A,Not assigned,Not assigned
10,M2B,Not assigned,Not assigned
15,M7B,Not assigned,Not assigned
...,...,...,...
174,M4Z,Not assigned,Not assigned
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned


#### Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.

#### Exporting the dataframe in csv file so that in 2nd question it can be used.

#### In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [10]:
joined_df.shape

(103, 3)

In [11]:
joined_df.to_csv('dataframe1.csv')